In [1]:
import os
import csv
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv')

In [3]:
hmda_vars = ['num_approved_loans', 
             'num_applications',
             'sum_approved_loans',
             'sum_all_loan_amount'
             #'median__all_income',
             #'sum_rate_spread',
             #'mean_LTV'
            ]

In [4]:
data = data[data["state"] == 6]

In [5]:
data['pct_minority'] = (data['Tot.Pop'] - data['Tot.WhitePop'])/data['Tot.Pop'] * 100

In [6]:
# finding the median income quintiles
# sort the median__all_income variable in ascending order
data.sort_values(by='pct_minority', inplace=True)

# divide the sorted median__all_income variable into 5 equal parts, or quintiles
data['pct_minority_Quintiles'] = pd.qcut(data['pct_minority'], q=5, labels=False)

# create a new variable called "Median_Income_Quintiles" based on the quintiles
data['pct_minority_Quintiles'] = pd.cut(data['pct_minority'], bins=5, labels=['20', '40', '60', '80', '100'])

In [7]:
Quintiles_loan_table = data.groupby('which_bank')[hmda_vars].mean()
#Quintiles_loan_table['loan_approvel_rate'] = Quintiles_loan_table['num_approved_loans']/Quintiles_loan_table['num_applications']
#Quintiles_loan_table = Quintiles_loan_table.sort_values(by='loan_approvel_rate', ascending=False)
Quintiles_loan_table

,num_approved_loans,num_applications,sum_approved_loans,sum_all_loan_amount
which_bank,,,,
All Other Banks,51.791350,61.556190,2.700681e+07,3.104069e+07
Bank of West,2.465273,3.490267,1.388032e+06,1.871864e+06


In [8]:
Quintiles_loan_table3 = data.groupby(['which_bank', "pct_minority_Quintiles"])[hmda_vars].sum()
Quintiles_loan_table3['loan_pct_in_minority_group'] = Quintiles_loan_table3['num_approved_loans']/Quintiles_loan_table3['num_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table3['$loan_pct_in_minority_group'] = Quintiles_loan_table3['sum_approved_loans']/Quintiles_loan_table3['sum_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table3['loan_approval_rate'] = Quintiles_loan_table3['num_approved_loans']/Quintiles_loan_table3['num_applications']
#Quintiles_loan_table3

In [9]:
Quintiles_loan_table3.unstack(level=0)

num_approved_loans              num_applications  \
which_bank                All Other Banks Bank of West  All Other Banks   
pct_minority_Quintiles                                                    
20                                  98601         2784           116528   
40                                 153481         3651           182401   
60                                  91986         1959           109877   
80                                  51394         1458            61507   
100                                 12858          406            14993   

                                    sum_approved_loans                \
which_bank             Bank of West    All Other Banks  Bank of West   
pct_minority_Quintiles                                                 
20                             3962       5.863134e+10  1.833340e+09   
40                             5105       7.796394e+10  2.037395e+09   
60                             2765       4.465607e+10  9.482750e+08   
80                             2111       2.485676e+10  7.251200e+08   
100                             580       6.812430e+09  2.314700e+08   

                       sum_all_loan_amount                \
which_bank                 All Other Banks  Bank of West   
pct_minority_Quintiles                                     
20                            6.714449e+10  2.478280e+09   
40                            8.935768e+10  2.706635e+09   
60                            5.138694e+10  1.277105e+09   
80                            2.907064e+10  9.965150e+08   
100                           7.763915e+09  3.302900e+08   

                       loan_pct_in_minority_group               \
which_bank                        All Other Banks Bank of West   
pct_minority_Quintiles                                           
20                                       0.241480     0.271398   
40                                       0.375884     0.355917   
60                                       0.225279     0.190973   
80                                       0.125867     0.142133   
100                                      0.031490     0.039579   

                       $loan_pct_in_minority_group               \
which_bank                         All Other Banks Bank of West   
pct_minority_Quintiles                                            
20                                        0.275367     0.317428   
40                                        0.366164     0.352759   
60                                        0.209731     0.164186   
80                                        0.116742     0.125549   
100                                       0.031995     0.040077   

                       loan_approval_rate               
which_bank                All Other Banks Bank of West  
pct_minority_Quintiles                                  
20                               0.846157     0.702675  
40                               0.841448     0.715181  
60                               0.837172     0.708499  
80                               0.835580     0.690668  
100                              0.857600     0.700000

In [10]:
data['pct_White'] = data['Tot.WhitePop']/data['Tot.Pop'] * 100
data['pct_Black'] = data['Tot.BlackPop ']/data['Tot.Pop'] * 100
data['pct_AmericIndian'] = data['Tot.AmericIndianPop']/data['Tot.Pop'] * 100
data['pct_Asian'] = data['Tot.AsianPop']/data['Tot.Pop'] * 100
data['pct_NativeHawaiianPacific'] = data['Tot.NativeHawaiianPacificPop']/data['Tot.Pop'] * 100
data['pct_OtherRaceAlone'] = data['Tot.OtherRaceAlonePop']/data['Tot.Pop'] * 100
data['pct_TwoOrMoreRace'] = data['Tot.TwoOrMoreRace']/data['Tot.Pop'] * 100
data['pct_TwoOrMoreRace(Some other race)'] = data['Tot.TwoOrMoreRace(Some other race)']/data['Tot.Pop'] * 100
data['pct_Hispanic/Latino'] = data['Tot.Hispanic/Latino']/data['Tot.Pop'] * 100
data['pct_NotHispanic/Latino'] = data['Tot.NotHispanic/Latino']/data['Tot.Pop'] * 100

In [11]:
data['major_minority'] = data[['pct_Black', 'pct_AmericIndian', 'pct_Asian','pct_NativeHawaiianPacific',
                                        'pct_OtherRaceAlone','pct_TwoOrMoreRace','pct_TwoOrMoreRace(Some other race)',
                                        'pct_Hispanic/Latino']].idxmax(axis=1)
data['major_minority'] = data['major_minority'].apply(lambda x: 'Black' if x=='pct_Black' 
                                                                        else 'American Indian' if x=='pct_AmericIndian' 
                                                                        else 'Asian' if x=='pct_Asian' 
                                                                        else 'Native Hawaiian Pacific' if x=='pct_NativeHawaiianPacific' 
                                                                        else 'Other Race Alone' if x=='pct_OtherRaceAlone' 
                                                                        else 'Two Or More Race' if x=='pct_TwoOrMoreRace' 
                                                                        else 'Two Or More Race(Some other race)' if x=='pct_TwoOrMoreRace(Some other race)' 
                                                                        else 'Hispanic/Latino')

In [12]:
Quintiles_loan_table4 = data.groupby(['which_bank', "major_minority"])[hmda_vars].sum()
Quintiles_loan_table4['loan_pct_in_minority_group'] = Quintiles_loan_table4['num_approved_loans']/Quintiles_loan_table4['num_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table4['$loan_pct_in_minority_group'] = Quintiles_loan_table4['sum_approved_loans']/Quintiles_loan_table4['sum_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table4['loan_approval_rate'] = Quintiles_loan_table4['num_approved_loans']/Quintiles_loan_table4['num_applications']
#Quintiles_loan_table4

In [13]:
Quintiles_loan_table4.unstack(level=0)

num_approved_loans              num_applications  \
which_bank                 All Other Banks Bank of West  All Other Banks   
major_minority                                                             
American Indian                      497.0          9.0            614.0   
Asian                             114107.0       3903.0         129526.0   
Black                               6913.0        183.0           8484.0   
Hispanic/Latino                   282196.0       5951.0         341292.0   
Native Hawaiian Pacific                9.0          NaN             10.0   
Other Race Alone                     116.0         12.0            145.0   
Two Or More Race                    4485.0        200.0           5238.0   

                                     sum_approved_loans                \
which_bank              Bank of West    All Other Banks  Bank of West   
major_minority                                                          
American Indian                 10.0       1.602250e+08  5.675000e+06   
Asian                         5592.0       7.782094e+10  2.810865e+09   
Black                          247.0       3.519875e+09  8.808500e+07   
Hispanic/Latino               8372.0       1.266412e+11  2.668265e+09   
Native Hawaiian Pacific          NaN       1.605000e+06           NaN   
Other Race Alone                19.0       1.037400e+08  6.390000e+06   
Two Or More Race               283.0       4.674075e+09  1.963200e+08   

                        sum_all_loan_amount                \
which_bank                  All Other Banks  Bank of West   
major_minority                                              
American Indian                1.914000e+08  5.870000e+06   
Asian                          8.665058e+10  3.835540e+09   
Black                          4.181220e+09  1.136550e+08   
Hispanic/Latino                1.482122e+11  3.552510e+09   
Native Hawaiian Pacific        1.960000e+06           NaN   
Other Race Alone               1.203850e+08  8.395000e+06   
Two Or More Race               5.367100e+09  2.728550e+08   

                        loan_pct_in_minority_group               \
which_bank                         All Other Banks Bank of West   
major_minority                                                    
American Indian                           0.001217     0.000877   
Asian                                     0.279453     0.380484   
Black                                     0.016930     0.017840   
Hispanic/Latino                           0.691110     0.580133   
Native Hawaiian Pacific                   0.000022          NaN   
Other Race Alone                          0.000284     0.001170   
Two Or More Race                          0.010984     0.019497   

                        $loan_pct_in_minority_group               \
which_bank                          All Other Banks Bank of West   
major_minority                                                     
American Indian                            0.000753     0.000983   
Asian                                      0.365491     0.486679   
Black                                      0.016531     0.015251   
Hispanic/Latino                            0.594778     0.461989   
Native Hawaiian Pacific                    0.000008          NaN   
Other Race Alone                           0.000487     0.001106   
Two Or More Race                           0.021952     0.033991   

                        loan_approval_rate               
which_bank                 All Other Banks Bank of West  
major_minority                                           
American Indian                   0.809446     0.900000  
Asian                             0.880958     0.697961  
Black                             0.814828     0.740891  
Hispanic/Latino                   0.826846     0.710822  
Native Hawaiian Pacific           0.900000          NaN  
Other Race Alone                  0.800000     0.631579  
Two Or More Race                  0.856243     